# Actividad para minimizar el impacto del sobre-ajuste.

## Etapa 1: Definición de los datos.

Antes de comenzar definimos la base de datos de interés. Usaremos la base de datos IMBD que consiste en reseñas hechas a 25000 películas. La tarea que configura esta base de datos consiste en predecir si el comentario es positivo o negativo. ([Ver enlace](https://keras.io/api/datasets/imdb/))

Se cargan las librerías necesarias

In [1]:
from keras.datasets import imdb
import numpy as np
from keras import models
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt

Se carga la base de datos

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

Se definen funciones para la visualización de la función de costo

In [3]:
def evaluate(model, history):
    # Se evalúa el accuracy del modelo tanto en el conjunto de entrenamiento como 
    # en el de prueba.
    _, train_accuracy = model.evaluate(x_train, y_train)
    _, test_accuracy = model.evaluate(x_test, y_test)

    print(f"Trainining accuracy: {train_accuracy:.2f}")
    print(f"Testing accuracy: {test_accuracy:.2f}")

    # Se grafica la función de costo para los conjuntos de entrenamiento y 
    # prueba.
    plt.figure(figsize=(4, 3), dpi=160)

    plt.plot(history.history["loss"], label="train")
    plt.plot(history.history["val_loss"], label="test")
    plt.legend()
    plt.show()

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [4]:
# Procesamiento de los datos.
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [5]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [6]:
# Se definen los conjuntos de entrenamiento y prueba
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [7]:
'''
# Se define la red neuronal.

model = models.Sequential([
  layers.Dense(16, activation='relu', input_shape=(10000,)),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
  ])
'''

"\n# Se define la red neuronal.\n\nmodel = models.Sequential([\n  layers.Dense(16, activation='relu', input_shape=(10000,)),\n  layers.Dense(16, activation='relu'),\n  layers.Dense(1, activation='sigmoid')\n  ])\n"

In [8]:
'''
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
'''

"\nmodel.compile(optimizer='rmsprop',\n              loss='binary_crossentropy',\n              metrics=['accuracy'])\n"

In [9]:
'''
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val))
'''

'\nhistory = model.fit(partial_x_train,\n                    partial_y_train,\n                    epochs=50,\n                    batch_size=512,\n                    validation_data=(x_val, y_val))\n'

In [10]:
#evaluate(model, history)

El modelo que entrenemos presenta sobre ajuste. Así, debemos construir una bitácora donde reportemos los siguientes análisis.  
 
Indicar los elementos que tuvimos en cuenta para verificar que el modelo inicial se encuentra en sobreajuste. 
Modificar el modelo inicial con el fin de minimizar el impacto del sobreajuste y reportar los resultados obtenidos en términos de la función de pérdida. En particular debemos experimentar las siguientes variaciones: 
* Modelo inicial sin ninguna * modificación. 
* Modelo inicial añadiendo * regularización L2. 
* Modelo inicial añadiendo dropout. 
* Modelo que combine ambas estrategias de regularización. 

**Juan Esteban Floyd y Juan David Aycardi**

# **BITACORA**

**Modelo inicial con ninguna modificacion:**

In [ ]:
model = models.Sequential([
  layers.Dense(16, activation='relu', input_shape=(10000,)),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
  ])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val))



Epoch 1/50
30/30 [==============================] - 7s 59ms/step - loss: 0.5602 - accuracy: 0.7723 - val_loss: 0.4425 - val_accuracy: 0.8530


In [ ]:
evaluate(model, history)

se puede evidenciar un sobreajuste ya que cuando se evalua el test con el modelo se ve que este esta muy alejado del valor de entrenamiento, cuando lo ideal es que estos esten lo mas cercano posible.
Se puede decir que el modelo se aprendio datos anómalos y por este mismo el modelo aprende patrones generales y estos no ayudan a la prediccion de valores validos.
En pocas palabras el modelo pierde la capacidad de generalizar.

**Modelo añadiendo regularizacion L2**

In [ ]:
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np

def build_model(regularization_rate):
  model = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(10000,), kernel_regularizer=regularizers.l2(regularization_rate)),
    layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(regularization_rate)),
    layers.Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = KerasClassifier(build_fn=build_model)
regularization_rates = [0.2]
param_grid = dict(regularization_rate=regularization_rates)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring="accuracy")
grid_result = grid.fit(partial_x_train, partial_y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))  

In [ ]:
# Usar el mejor modelo encontrado para entrenar y evaluar
best_model = build_model(grid_result.best_params_['regularization_rate'])
history = best_model.fit(partial_x_train,
                           partial_y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(x_val, y_val))

In [ ]:
evaluate(best_model, history)

Se puede ver que añadiendo un valor en la regularizacion de x=0.2 el modelo tiene un compartamiento mucho mas parecido al esperado. Es decir, que el resultado del entrenamiento y el test sean lo mas parecido posible.

**Modelo inicial añadiendo dropout**

In [ ]:
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np

def build_model(dropout):
  model = Sequential([
    layers.Dropout(dropout),
    layers.Dense(16, activation='relu', input_shape=(10000,)),
    layers.Dropout(dropout),
    layers.Dense(16, activation='relu'),
    layers.Dropout(dropout),
    layers.Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = KerasClassifier(build_fn=build_model)
dropouts = [0.53]
param_grid = dict(dropout = dropouts)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring="accuracy")
grid_result = grid.fit(partial_x_train, partial_y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))  

In [ ]:
# Usar el mejor modelo encontrado para entrenar y evaluar
best_model = build_model(grid_result.best_params_['dropout'])
history = best_model.fit(partial_x_train,
                           partial_y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(x_val, y_val))

In [ ]:
evaluate(best_model, history)

Con una busqueda de parametros en el dropout de 0.4 se ve que tiende a normalizarse pero sigue estando muy alejado del valor esperado, es decir, sigue siendo mejor opcion la de aplicar regularizacion

**Modelo que combina ambas**

In [ ]:
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np

def build_model(dropout, regularization_rate):
  model = models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(10000,), kernel_regularizer=regularizers.l2(regularization_rate)),
    layers.Dropout(dropout),
    layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(regularization_rate)),
    layers.Dropout(dropout),
    layers.Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = KerasClassifier(build_fn=build_model)
regularization_rates = [0.145]
dropouts = [0.4]
param_grid = dict(regularization_rate=regularization_rates, dropout = dropouts)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring="accuracy")
grid_result = grid.fit(partial_x_train, partial_y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))  

In [ ]:
# Usar el mejor modelo encontrado para entrenar y evaluar
best_model = build_model(dropout=grid_result.best_params_['dropout'],
                          regularization_rate=grid_result.best_params_['regularization_rate'])
history = best_model.fit(partial_x_train,
                         partial_y_train,
                         epochs=50,
                         batch_size=32,
                         validation_data=(x_val, y_val))

In [ ]:
evaluate(best_model, history)

Se puede ver que con un valor de regularizacion = 0.145 y con un valor de dropout = 0.4 la grafica tiende a tener un comportamiento como lo esperado, pero este no es tan bueno commo el de la regularizacion solo, es decir. Si bien se encuentra un buen valor, el compartamiento que se obtiene en esta no es la suficientemente bueno en comparacion a las diferentes alternativas.

**Conclusiones:**

Gracias a las formas que se vieron el clase de prevenir un sobreajuste se ve que ayuda bastante en su proposito ya que en el modelo se ven comportamientos parecidos en el entranamiento y en el test. Esto hace que el modelo tengo una buena capacidad generalizar. 
Es bastante efectivo el usar regularizacion y dropout pero tambien se videncio dificultades tecnicas y esto hace que debido a la limitaciones de recursos sea dificil de hacer una busqueda de hiperparamtetros y asi mismo, encontrar el mejor de estos mismos.